# almond

In [1]:
val a = 1
val b = 2
a + b

a: Int = 1
b: Int = 2
res1_2: Int = 3

In [2]:
// string interpolation
s"a=${a}"

res2: String = "a=1"

# Ammonite API

# Almond Jupyter API

## High level API

In [3]:
// These can be used to display things straightaway
Html("<b>Bold</b>")

Bold

In [4]:
// A handle can also be retained, to later update or clear things
val handle = Markdown("""
# title
## section
text
""")

// can be updated in later cells
// (this updates the previous display)
handle.withContent("""
# updated title
## new section
_content_
""").update()

// can be later cleared
handle.clear()


# title
## section
text


In [4]:
//  NOT WORKING!!!
// // Request input from user
// val result = Input().request()

// val result = Input().withPrompt(">>> ").request()

// Request input via password field
// val result = Input().withPassword().request()

## Request input

In [4]:
// NOT WORKING
// kernel.stdin() // clear text input

In [4]:
// final class Password(val value: String) {
//   override def toString = "****"
// }

// val user = kernel.stdin("user: ")
// val password = new Password(kernel.stdin(prompt = ">> ", password = true))

## Comm messages

In [4]:
// NOT WORKING!!!
// val id = java.util.UUID.randomUUID().toString
// kernel.publish.html("Waiting", id)

// kernel.comm.receiver("A") { data =>
//   // received message `data` from front-end
//   kernel.publish.updateHtml(s"<code>$data</code>", id)
// }

In [4]:
// kernel.publish.js("""
// // runs in the front-end(browser)
// var comm = Jupyter.notebook.kernel.comm_manager.new_comm("A", {"a": 2, "b": false});
// var message = prompt("Message")
// comm.send({"content": message});
// """)

## Display data

In [5]:
// val dd = almond.interpreter.api.DisplayData()
// // dd.add("application/myapp+json", """{"a": "A"}""")
// dd.add("text/html","<b>A</b>")
// kernel.publish.display(dd)

kernel.publish.display(
  almond.interpreter.api.DisplayData(
    Map(
      // if we set up an extension for application/myapp+json, first element should be picked
      "application/myapp+json" -> """{"a": "A"}""",
      // else, text/html should be displayed
      "text/html" -> "<b>A</b>"
    )
  )
)

cmd5.sc:7: method apply in object DisplayData is deprecated (since 0.14.2): Prefer the constructor with no argument, then calling add and addStringifiedJson on it
  almond.interpreter.api.DisplayData(
                         ^


A

In [6]:
for (i <- 1 to 10) {
  kernel.publish.html(s"Got item <b>#$i</b>")
  Thread.sleep((200.0 + 200.0 * scala.util.Random.nextGaussian).toLong max 0L)
}

Got item #1

Got item #2

Got item #3

Got item #4

Got item #5

Got item #6

Got item #7

Got item #8

Got item #9

Got item #10

## Updatable display data

In [7]:
val id = java.util.UUID.randomUUID().toString
kernel.publish.html("Starting", id)

for (i <- 1 to 10) {
  Thread.sleep((200.0 + 200.0 * scala.util.Random.nextGaussian).toLong max 0L)
  kernel.publish.updateHtml(s"Got item <b>#$i</b>", id)
}

kernel.publish.updateHtml("Got all items", id)

Got all items

id: String = "3a2660a7-3eac-431f-9ee3-0f94d034c3f9"

## After Interrupt Hooks

In [7]:
// val sparkAmmonite = {
//   new AmmoniteSparkSessionBuilder()
//     .getOrCreate()
// }
// lazy val spark = {
//   NotebookSparkSession.builder()
//     .progress(enable=true, keep=false)
//     .logsInDeveloperConsole(false)
//     .getOrCreate()
// }
// lazy val sc = {
//   spark.sparkContext
// }

// // Add new hook with name "CancelAllSparkJobs"
// kernel.addPostInterruptHook(
//   "CancelAllSparkJobs",
//   _ => sc.cancelAllJobs()
// )

// // Return a list with all registered hooks
// kernel.postInterruptHooks

// // Remove hook by name
// kernel.removePostInterruptHook("CancelAllSparkJobs")

// // Run after-interrupt hooks (called internally after a cell interrupt)
// kernel.runPostInterruptHooks()

## Hooks

In [8]:
private def runSql(sql: String): String = {
  println("Running query...")
  val fakeResult =
    """<table>
      |<tr>
      |<th>Id</th>
      |<th>Name</th>
      |</tr>
      |<tr>
      |<td>1</td>
      |<td>Tree</td>
      |</tr>
      |<tr>
      |<td>2</td>
      |<td>Apple</td>
      |</tr>
      |</table>
      |""".stripMargin
  fakeResult
}

kernel.addExecuteHook { code =>
  import almond.api.JupyterApi
  import almond.interpreter.api.DisplayData

  if (code.linesIterator.take(1).toList == List("%sql")) {
    val sql = code.linesWithSeparators.drop(1).mkString // drop first line with "%sql"
    val result = runSql(sql)
    Left(JupyterApi.ExecuteHookResult.Success(DisplayData.html(result)))
  }
  else
    Right(code) // just pass on code
}

res8_1: Boolean = true

In [8]:
%sql
SELECT id, name FROM my_table

Running query...


Id,Name
1,Tree
2,Apple


## Image

In [9]:
import almond.api.helpers.Display.{Image => IMAGE}
IMAGE.fromUrl("https://scala-lang.org/resources/img/frontpage/scala-spiral.png", width=Some("50"))

cmd9.sc:2: object Image in object Display is deprecated (since 0.4.1): Use almond.display.Image instead
val res9_1 = IMAGE.fromUrl("https://scala-lang.org/resources/img/frontpage/scala-spiral.png", width=Some("50"))
             ^
cmd9.sc:16: class Display in package helpers is deprecated (since 0.4.1): Use almond.display.Data instead
          .print(res9_1, "res9_1", _root_.scala.None)
                ^


import almond.api.helpers.Display.{Image => IMAGE}
res9_1: almond.api.helpers.Display = image/png #fed3d2a0-8085-4b31-8ee8-9b965afc8bb2

## Javascript

In [10]:
// in browser
Javascript("""
    console.log("almond!!!");
""")

In [11]:
Html("""<div id="vis"></div>""")

In [12]:
val jsonData = """
{
  "$schema": "https://vega.github.io/schema/vega/v6.json",
  "description": "A basic bar chart example, with value labels shown upon pointer hover.",
  "width": 400,
  "height": 200,
  "padding": 5,
  "data": [
    {
      "name": "table",
      "values": [
        {
          "category": "A",
          "amount": 28
        },
        {
          "category": "B",
          "amount": 55
        },
        {
          "category": "C",
          "amount": 43
        },
        {
          "category": "D",
          "amount": 91
        },
        {
          "category": "E",
          "amount": 81
        },
        {
          "category": "F",
          "amount": 53
        },
        {
          "category": "G",
          "amount": 19
        },
        {
          "category": "H",
          "amount": 87
        }
      ]
    }
  ],
  "signals": [
    {
      "name": "tooltip",
      "value": {},
      "on": [
        {
          "events": "rect:pointerover",
          "update": "datum"
        },
        {
          "events": "rect:pointerout",
          "update": "{}"
        }
      ]
    }
  ],
  "scales": [
    {
      "name": "xscale",
      "type": "band",
      "domain": {
        "data": "table",
        "field": "category"
      },
      "range": "width",
      "padding": 0.05,
      "round": true
    },
    {
      "name": "yscale",
      "domain": {
        "data": "table",
        "field": "amount"
      },
      "nice": true,
      "range": "height"
    }
  ],
  "axes": [
    {
      "orient": "bottom",
      "scale": "xscale"
    },
    {
      "orient": "left",
      "scale": "yscale"
    }
  ],
  "marks": [
    {
      "type": "rect",
      "from": {
        "data": "table"
      },
      "encode": {
        "enter": {
          "x": {
            "scale": "xscale",
            "field": "category"
          },
          "width": {
            "scale": "xscale",
            "band": 1
          },
          "y": {
            "scale": "yscale",
            "field": "amount"
          },
          "y2": {
            "scale": "yscale",
            "value": 0
          }
        },
        "update": {
          "fill": {
            "value": "steelblue"
          }
        },
        "hover": {
          "fill": {
            "value": "red"
          }
        }
      }
    },
    {
      "type": "text",
      "encode": {
        "enter": {
          "align": {
            "value": "center"
          },
          "baseline": {
            "value": "bottom"
          },
          "fill": {
            "value": "#333"
          }
        },
        "update": {
          "x": {
            "scale": "xscale",
            "signal": "tooltip.category",
            "band": 0.5
          },
          "y": {
            "scale": "yscale",
            "signal": "tooltip.amount",
            "offset": -2
          },
          "text": {
            "signal": "tooltip.amount"
          },
          "fillOpacity": [
            {
              "test": "datum === tooltip",
              "value": 0
            },
            {
              "value": 1
            }
          ]
        }
      }
    }
  ]
}
"""

jsonData: String = """
{
  "$schema": "https://vega.github.io/schema/vega/v6.json",
  "description": "A basic bar chart example, with value labels shown upon pointer hover.",
  "width": 400,
  "height": 200,
  "padding": 5,
  "data": [
    {
      "name": "table",
      "values": [
        {
          "category": "A",
          "amount": 28
        },
        {
          "category": "B",
          "amount": 55
        },
        {
          "category": "C",
          "amount": 43
        },
        {
          "category": "D",
          "amount": 91
        },
        {
          "category": "E",
          "amount": 81
        },
        {
          "category": "F",
          "amount": 53
        },
        {
          "category": "G",
          "amount": 19
...

In [13]:
// in browser
// NOT WORKING in VSCode!!!

// lib
Javascript("""
var script = document.createElement('script'); 
script.type = 'text/javascript'; 
script.src = 'https://cdn.jsdelivr.net/npm/vega@6'; 
document.head.appendChild(script); 

var script = document.createElement('script'); 
script.type = 'text/javascript'; 
script.src = 'https://cdn.jsdelivr.net/npm/vega-lite@6'; 
document.head.appendChild(script); 

var script = document.createElement('script'); 
script.type = 'text/javascript'; 
script.src = 'https://cdn.jsdelivr.net/npm/vega-embed@6'; 
document.head.appendChild(script); 
""")

// data
// import $ivy.`com.lihaoyi::ujson:4.0.2`
import $ivy.`com.lihaoyi::upickle:4.3.2`
// import $ivy.`com.lihaoyi::requests:0.9.0`
// val spec = ujson.read(requests.get("https://vega.github.io/vega/examples/bar-chart.vg.json", verifySslCerts = false).text())
val spec = ujson.read(jsonData)

spec("data")(0)("values") = ujson.Arr(
  ujson.Obj("category" -> "Epic", "amount" -> 50),
  ujson.Obj("category" -> "amounts", "amount" -> 100)
)
print(s""" vegaEmbed("#vis", $spec).then(function(result) { }).catch(console.error) """)
// display
Javascript(s""" vegaEmbed("#vis", $spec).then(function(result) { }).catch(console.error) """)

 vegaEmbed("#vis", {"$schema":"https://vega.github.io/schema/vega/v6.json","description":"A basic bar chart example, with value labels shown upon pointer hover.","width":400,"height":200,"padding":5,"data":[{"name":"table","values":[{"category":"Epic","amount":50},{"category":"amounts","amount":100}]}],"signals":[{"name":"tooltip","value":{},"on":[{"events":"rect:pointerover","update":"datum"},{"events":"rect:pointerout","update":"{}"}]}],"scales":[{"name":"xscale","type":"band","domain":{"data":"table","field":"category"},"range":"width","padding":0.05,"round":true},{"name":"yscale","domain":{"data":"table","field":"amount"},"nice":true,"range":"height"}],"axes":[{"orient":"bottom","scale":"xscale"},{"orient":"left","scale":"yscale"}],"marks":[{"type":"rect","from":{"data":"table"},"encode":{"enter":{"x":{"scale":"xscale","field":"category"},"width":{"scale":"xscale","band":1},"y":{"scale":"yscale","field":"amount"},"y2":{"scale":"yscale","value":0}},"update":{"fill":{"value":"steelbl

import $ivy.$
spec: ujson.Value.Value = Obj(
  Map(
    "$schema" -> Str("https://vega.github.io/schema/vega/v6.json"),
    "description" -> Str(
      "A basic bar chart example, with value labels shown upon pointer hover."
    ),
    "width" -> Num(400.0),
    "height" -> Num(200.0),
    "padding" -> Num(5.0),
    "data" -> Arr(
      ArrayBuffer(
        Obj(
          Map(
            "name" -> Str("table"),
            "values" -> Arr(
              ArrayBuffer(
                Obj(Map("category" -> Str("Epic"), "amount" -> Num(50.0))),
                Obj(Map("category" -> Str("amounts"), "amount" -> Num(100.0)))
              )
            )
          )
        )
      )
    ),
    "signals" -> Arr(
      ArrayBuffer(
        Obj(
          Map(
            "name" -> Str("tooltip"),
            "value" -> Obj(Map()),
            "on" -> Arr(
              ArrayBuffer(
                Obj(
                  Map(
                    "events" -> Str("rect:pointerover"),
           